<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/stuff/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase paho-mqtt nltk ipywidgets --quiet
!pip install ipywidgets

In [ ]:

import json
import time
import re
import threading
import paho.mqtt.client as mqtt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from firebase import firebase as fb
from collections import defaultdict
from nltk.stem import PorterStemmer
import ipywidgets as widgets
from IPython.display import display, clear_output,HTML
import pandas as pd
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas





# התחברות לפיירבייס
FIREBASE_URL = 'https://shabloolsearchengine-default-rtdb.asia-southeast1.firebasedatabase.app/'
FBconn = fb.FirebaseApplication(FIREBASE_URL, None)
print('✅ Connected to Firebase')

# הגדרת stemmer ו־stop words
stemmer = PorterStemmer()
custom_stop_words = {
    'a', 'an', 'the', 'and', 'or', 'but',
    'to', 'from', 'of', 'in', 'on', 'at', 'for', 'by', 'with', 'as',
    'this', 'that', 'these', 'those', 'it', 'its',
    'be', 'is', 'are', 'was', 'were', 'has', 'have', 'had',
    'not', 'no', 'yes', 'true', 'false', 'null', 'none'
}
loggedin_user = None # Global variable to hold currently logged in user
loggedin_user_rank = None # Global variable to hold currently logged in users rank


In [ ]:
def search_mqtt(query, num_results=5):
    index = FBconn.get('/search_index', None)
    if not index:
        return []

    query_words = [stemmer.stem(w.lower()) for w in re.findall(r'\w+', query)]
    if not query_words:
        return []

    page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})
    for term in query_words:
        if term in index:
            for doc_id in index[term]['DocIDs']:
                page_scores[doc_id]['matches'] += 1
                page_scores[doc_id]['total_freq'] += 1

    ranked = sorted([
        (doc_id, score['matches'], score['total_freq'])
        for doc_id, score in page_scores.items()
    ], key=lambda x: (x[1], x[2]), reverse=True)

    return [
        {'doc_id': doc_id, 'matching_words': matches, 'total_frequency': total_freq}
        for doc_id, matches, total_freq in ranked[:num_results]
    ]


In [ ]:
def process_message(data):
    timestamp = str(int(time.time() * 1000))
    FBconn.put('/sensor_data', timestamp, data)

def on_message(client, userdata, msg):
    try:
        payload = json.loads(msg.payload.decode())
        process_message(payload)
        print(f"✅ Saved sensor data: {payload}")
    except json.JSONDecodeError:
        print("❌ Invalid JSON received")

def start_mqtt():
    client = mqtt.Client()
    client.on_message = on_message
    client.connect("test.mosquitto.org", 1883, 60)
    client.subscribe("braude/D106/indoor")
    client.subscribe("braude/D106/outdoor")
    client.loop_start()
    print("📡 Listening for sensor data (MQTT)...")
    time.sleep(60)
    client.loop_stop()
    print("✅ Finished listening")


In [ ]:
start_mqtt()

In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import re

base_url = "https://mqtt.org"
start_page = base_url + "/"

index = defaultdict(set)  # term -> set of page IDs
doc_titles = {}           # page ID -> page title

def clean_and_tokenize(text):
    words = re.findall(r'\b[a-zA-Z]{2,}\b', text.lower())
    return [stemmer.stem(word) for word in words if word not in custom_stop_words]

# שלב 1: שליפת כל הקישורים הפנימיים מהעמוד הראשי
try:
    response = requests.get(start_page)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a', href=True)

    pages = {}
    page_id = 1
    for link in links:
        href = link['href']
        if href.startswith('/'):  # רק קישורים פנימיים
            full_url = base_url + href
            if full_url not in pages.values():
                pages[f"page{page_id}"] = full_url
                page_id += 1

except Exception as e:
    print(f"❌ Failed to crawl links from {start_page}: {e}")
    pages = {}

# שלב 2: אינדוקס הדפים שנמצאו
for doc_id, url in pages.items():
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text()
        words = clean_and_tokenize(text)

        for word in words:
            index[word].add(doc_id)

        title = soup.title.string.strip() if soup.title and soup.title.string else url
        doc_titles[doc_id] = title

        print(f"✅ Indexed {doc_id}: {title}")
    except Exception as e:
        print(f"❌ Failed to load {url}: {e}")

# המרה ל-list לקראת העלאה
index = {term: list(doc_ids) for term, doc_ids in index.items()}

# שלב 3: העלאה לפיירבייס
FBconn.put('/', 'search_index', index)
FBconn.put('/', 'doc_titles', doc_titles)
print("✅ Crawled pages uploaded to Firebase.")


In [ ]:

firebase_ready_index = {
    term: {"term": term, "DocIDs": docs}
    for term, docs in index.items()
}


FBconn.put('/', 'search_index', firebase_ready_index)
print("✅ Index uploaded to Firebase (single request)")


In [ ]:
def fetch_index():
    raw = FBconn.get('/search_index', None)
    if not raw:
        return {}
    return {term: data['DocIDs'] for term, data in raw.items() if 'DocIDs' in data}


In [ ]:
def logout_user():
    global loggedin_user
    global loggedin_user_rank
    loggedin_user = None
    loggedin_user_rank = None
    print("Logged out.")
    main_dashboard()

In [ ]:
def employee_screen():
  with content_output:
    content_output.clear_output()

In [ ]:
def user_screen():
    with content_output:
        content_output.clear_output()
        global loggedin_user
        global loggedin_user_rank
        if loggedin_user_rank is not None:
            rank = loggedin_user_rank.lower()
            if rank == 'manager':
                admin_screen()
                return
            elif rank == 'employee':
                employee_screen()
                return

        # Widgets for login
        username = widgets.Text(
            description='Username:',
            placeholder='Enter username',
            layout=widgets.Layout(width='300px')
        )
        password = widgets.Password(
            description='Password:',
            placeholder='Enter password',
            layout=widgets.Layout(width='300px')
        )
        login_button = widgets.Button(
            description='Login',
            button_style='success',
            layout=widgets.Layout(width='100px')
        )
        message = widgets.HTML(value="")  # To display messages (error or success)

        # Login handler function
        def on_login_clicked(b):
          global loggedin_user
          global loggedin_user_rank
          message.value = ""

          user = username.value.strip()
          pwd = password.value.strip()

          # Pull user data from Firebase
          user_data = FBconn.get(f'/Users/{user}', None)

          if user_data is None:
              message.value = "<p style='color:red;'>❌ User does not exist.</p>"
              return

          # Validate password
          if user_data.get('password') == pwd:
              loggedin_user = user  # ✅ Set logged-in user
              rank = user_data.get('rank', 'unknown')
              loggedin_user_rank = rank
              message.value = f"<p style='color:green;'>✅ Login successful! Welcome, {user} ({rank})</p>"

              # Return to Rerender Main Screen
              main_dashboard()
          else:
              message.value = "<p style='color:red;'>❌ Incorrect password.</p>"

        login_button.on_click(on_login_clicked)

        # Layout the login form
        form = widgets.VBox([username, password, login_button, message],
                            layout=widgets.Layout(
                                align_items='center',
                                border='1px solid gray',
                                padding='20px',
                                width='350px'
                            ))

        display(form)


In [ ]:
def search_screen():
    with content_output:
        content_output.clear_output()

        search_input = widgets.Text(
            placeholder='Enter term...',
            description='Search:',
            layout=widgets.Layout(width='70%'),
            style={'description_width': 'initial'}
        )
        output = widgets.Output()

        def do_search(b):
            query = search_input.value.strip()
            if not query:
                with output:
                    output.clear_output()
                    display(HTML("<p style='color:red;'>Please enter a search term.</p>"))
                return

            results = search_mqtt(query)
            doc_titles = FBconn.get('/doc_titles', None) or {}

            with output:
                output.clear_output()
                if not results:
                    display(HTML(f"<p style='color:red;'>❌ No results found for: <b>{query}</b></p>"))
                else:
                    html = f"<p style='color:green;'>✅ {len(results)} result(s) for <b>{query}</b></p><ul style='line-height:1.8;'>"
                    for res in results:
                        doc_id = res['doc_id']
                        title = doc_titles.get(doc_id, 'Unknown Title')
                        html += f"<li><b>{doc_id}</b> — <span style='color:#333;'>{title}</span> — Matches: <b>{res['matching_words']}</b>, Frequency: <b>{res['total_frequency']}</b></li>"
                    html += "</ul>"
                    display(HTML(html))

        search_btn = widgets.Button(description="Search", button_style='primary', layout=widgets.Layout(width='30%'))
        search_btn.on_click(do_search)

        title = "<h2 style='color:#0066cc;'>🔍 Shablool Search Engine</h2><hr>"
        display(HTML(title))
        display(widgets.VBox([
            search_input,
            widgets.HBox([search_btn]),
            output
        ]))


In [ ]:
def stats_screen():
    with content_output:
        content_output.clear_output()

        # Fetch data from Firebase
        raw = FBconn.get('/sensor_data', None)
        if not raw:
            display(HTML("<p style='color:red;'>❌ No sensor data available.</p>"))
            return

        # Prepare DataFrame
        df = pd.DataFrame(raw).T

        # Attempt to convert index to numeric (timestamps), ignore non-numeric
        df.index = pd.to_numeric(df.index, errors='coerce')
        df = df.loc[~df.index.isna()]
        df.index = pd.to_datetime(df.index, unit='ms')

        # Sort by datetime index
        df = df.sort_index()

        df = df.sort_index()

        # Ensure relevant sensor columns exist
        sensors = ['Humidity', 'Pressure', 'Temperature', 'Dlight', 'Distance']
        available_sensors = [s for s in sensors if s in df.columns]

        if not available_sensors:
            display(HTML("<p style='color:red;'>❌ No relevant sensor columns found.</p>"))
            return

        # Create dropdown widgets
        sensor_dropdown = widgets.Dropdown(
            options=available_sensors,
            description='Sensor:',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='250px')
        )

        chart_dropdown = widgets.Dropdown(
            options=['Line Plot', 'Histogram'],
            description='Chart:',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='250px')
        )

        out = widgets.Output()

        # Define redraw function here, with access to df
        def redraw(change=None):
            with out:
                out.clear_output()
                sensor = sensor_dropdown.value
                chart_type = chart_dropdown.value

                try:
                    valid_mask = df[sensor].notna()
                    data = df.loc[valid_mask, sensor].astype(float)
                    timestamps = df.index[valid_mask]
                except Exception:
                    display(HTML("<p style='color:red;'>❌ Data for selected sensor is invalid.</p>"))
                    return

                if chart_type == 'Line Plot':
                    fig, ax = plt.subplots(figsize=(10, 4))
                    ax.plot(timestamps, data, marker='o', linestyle='-', color='orange')
                    ax.set_title(f"{sensor} Over Time")
                    ax.set_xlabel("Timestamp")
                    ax.set_ylabel(sensor)
                    ax.grid(True)
                    fig.autofmt_xdate(rotation=30)
                    fig.tight_layout()
                    display(fig)
                    plt.close(fig)

                elif chart_type == 'Histogram':
                    fig, ax = plt.subplots(figsize=(8, 4))
                    ax.hist(data, bins=30, color='skyblue', edgecolor='black')
                    ax.set_title(f"{sensor} Distribution")
                    ax.set_xlabel(sensor)
                    ax.set_ylabel("Frequency")
                    fig.tight_layout()
                    display(fig)
                    plt.close(fig)

        # Connect the dropdowns to redraw
        sensor_dropdown.observe(redraw, names='value')
        chart_dropdown.observe(redraw, names='value')

        # Display widgets and output
        display(widgets.VBox([sensor_dropdown, chart_dropdown, out]))

        # Initial call to show the first plot
        redraw()


In [ ]:
def bonus_feature():
  with content_output:
    content_output.clear_output()
    data = fetch_index()
    if not data:
        display(HTML("<p style='color:red;'>❌ No index data available.</p>"))
        return

    most_common = max(data.items(), key=lambda item: len(item[1]))
    term, docs = most_common

    html = f"""
    <h2 style='color:#0066cc;'>🌟 Bonus Feature</h2>
    <p>Most frequent term:</p>
    <h3 style='color:green;'>“{term}”</h3>
    <p>Appears in <b>{len(docs)}</b> pages.</p>
    <hr>
    """
    display(HTML(html))





In [ ]:
def admin_screen():
  with content_output:
    content_output.clear_output()

    html = """
    <div style='text-align: center;'>
        <h2 style='color: #0066cc;'>Admin Dashboard</h2>
    </div>
    """
    display(HTML(html))

    # Define buttons
    btn_Register = widgets.Button(description="Register a New User", button_style='success', layout=widgets.Layout(width='240px'))
    btn_Credit = widgets.Button(description="Give Extra points to user", button_style='primary', layout=widgets.Layout(width='240px'))
    btn_leaderBoard = widgets.Button(description="Watch LeaderBoard", button_style='info', layout=widgets.Layout(width='240px'))

    # Button click handlers render to content_output
    btn_Register.on_click(lambda b: register_screen())
    btn_Credit.on_click(lambda b: search_screen())
    btn_leaderBoard.on_click(lambda b: stats_screen())

    display(widgets.HBox([btn_Register, btn_Credit, btn_leaderBoard],
                              layout=widgets.Layout(justify_content='center')))


In [ ]:
def firebase_register(username, password, rank):
    FBconn.put('/Users', username, {
        'password': password,
        'rank': rank
    })
    print(f"User '{username}' registered successfully.")


In [ ]:
def register_screen():
    with content_output:
        content_output.clear_output()
        html = """
        <div style='text-align: center;'>
            <h2 style='color:#0066cc;'>Register a new User</h2>
        </div>
        """
        display(HTML(html))

        input_layout = widgets.Layout(width='400px')
        description_style = {'description_width': '100px'}

        username = widgets.Text(
            description='Username:',
            placeholder='Enter username',
            layout=input_layout,
            style=description_style
        )
        password = widgets.Password(
            description='Password:',
            placeholder='Enter password',
            layout=input_layout,
            style=description_style
        )
        rank_dropdown = widgets.Dropdown(
            options=['Manager', 'Employee'],
            description='Rank:',
            layout=input_layout,
            style=description_style
        )

        register_button = widgets.Button(
            description='Register',
            button_style='success',
            layout=widgets.Layout(width='175px')
        )
        btn_back = widgets.Button(
            description="Back",
            button_style='info',
            layout=widgets.Layout(width='175px')
        )
        message = widgets.HTML(value="")

        def on_register_clicked(b):
            uname = username.value.strip()
            pwd = password.value.strip()
            rank = rank_dropdown.value

            if not uname or not pwd:
                message.value = "<span style='color:red;'>Username and password are required.</span>"
                return

            firebase_register(uname, pwd, rank)
            message.value = "<span style='color:green;'>User registered successfully.</span>"

        register_button.on_click(on_register_clicked)
        btn_back.on_click(lambda b: admin_screen())

        form = widgets.VBox([username, password, rank_dropdown, message])
        centered_buttons = widgets.HBox([register_button, btn_back], layout=widgets.Layout(justify_content='center'))

        display(widgets.VBox([form, centered_buttons]))



In [ ]:

# Create one global output area for all screen content
content_output = widgets.Output()

def main_dashboard():
    clear_output(wait=True)
    content_output.clear_output()


    # Title (kept outside the fixed-size window)
    title_html = """
    <h1 style="text-align:center; color:#0066cc;">Shablool Team</h1>
    <hr>
    <p style="text-align:center;">Choose an action:</p>
    """
    display(HTML(title_html))

    # Define buttons
    btn_user = widgets.Button(description="🛠️ User", button_style='warning', layout=widgets.Layout(width='120px'))
    btn_search = widgets.Button(description="🔍 Search", button_style='primary', layout=widgets.Layout(width='120px'))
    btn_stats = widgets.Button(description="📊 Stats", button_style='success', layout=widgets.Layout(width='120px'))
    btn_bonus = widgets.Button(description="🌟 Bonus", button_style='info', layout=widgets.Layout(width='120px'))

    # Button click handlers render to content_output
    btn_user.on_click(lambda b: user_screen())
    btn_search.on_click(lambda b: search_screen())
    btn_stats.on_click(lambda b: stats_screen())
    btn_bonus.on_click(lambda b: bonus_feature())
    buttons = [btn_user, btn_search, btn_stats, btn_bonus]


    if loggedin_user is not None:
      btn_profile = widgets.Button(description="👤 LogOut", button_style='danger', layout=widgets.Layout(width='120px'))
      btn_profile.on_click(lambda b: logout_user())  # Define this function if needed
      buttons.append(btn_profile)

    # Button bar
    button_bar = widgets.HBox(buttons,
                          layout=widgets.Layout(justify_content='center'))



    if loggedin_user is not None:
      print("Welcome, " + loggedin_user + " (" + loggedin_user_rank + ")")

    # Fixed-size container for buttons + content
    window = widgets.VBox(
        [button_bar, content_output],
        layout=widgets.Layout(
            width='100%',
            height='600px',
            border='2px solid gray',
            padding='10px',
            overflow='auto',
            align_items='center',
        )
    )

    display(window)


In [ ]:
main_dashboard()
